# ExperimentAllPair
### Take the dataset where the negatives are comprised of enhancers and random genomic data from the four controls (length-repeats, length-no_repeats, length-gc-repeats, length-gc-no_repeats).
### Train a single separable convolutional classifier on pairs and evaluate

## Imports

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
# # Credit: https://stackoverflow.com/questions/34199233/how-to-prevent-tensorflow-from-allocating-the-totality-of-a-gpu-memory
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [2]:
import data
from tensorflow import keras
import nets
from loaders import PremadePairSequence

## Files

In [3]:
# file_name = '/home/zakaria/Data/IdentityData/ShortTrain/60.fa'#file_name = '../Data/sequences.fa'

data_dir = '../Data/Datasets/All/'


train_sequence = f'{data_dir}/train_sequences.fa'
valid_sequence = f'{data_dir}/valid_sequences.fa'
test_sequence = f'{data_dir}/test_sequences.fa'

reverse_train_sequence = f'{data_dir}/reverse_train_sequences.fa'
reverse_valid_sequence = f'{data_dir}/reverse_valid_sequences.fa'
reverse_test_sequence = f'{data_dir}/reverse_test_sequences.fa'

train_triplet_dis = f'{data_dir}/train_triplet_dis.npy'
valid_triplet_dis = f'{data_dir}/valid_triplet_dis.npy'
test_triplet_dis = f'{data_dir}/test_triplet_dis.npy'

train_triplet_sim = f'{data_dir}/train_triplet_sim.npy'
valid_triplet_sim = f'{data_dir}/valid_triplet_sim.npy'
test_triplet_sim = f'{data_dir}/test_triplet_sim.npy'

## Parameters

In [4]:
codings_size=100 
exp_filter_num=4
exp_filter_1d_size= 11

allow_reverse = True
max_len = 600

learning_rate = 0.1
momentum = 0.9
is_nesterov = True
patience = 20
monitor = 'val_crm_f1_score'
start_from_epoch = 10

## Data

In [5]:
train = data.FantomToOneHotConverter(train_sequence, 0, max_len).seq_matrix
reverse_train = data.FantomToOneHotConverter(reverse_train_sequence, 0, max_len).seq_matrix if allow_reverse else None

In [6]:
valid = data.FantomToOneHotConverter(valid_sequence, 0, max_len).seq_matrix
reverse_valid = data.FantomToOneHotConverter(reverse_valid_sequence, 0, max_len).seq_matrix if allow_reverse else None

In [7]:
test = data.FantomToOneHotConverter(test_sequence, 0, max_len).seq_matrix
reverse_test = data.FantomToOneHotConverter(reverse_test_sequence, 0, max_len).seq_matrix if allow_reverse else None

In [8]:
train_seq = PremadePairSequence(train, train_triplet_dis, batch_size = 1024, reverse_x_in=reverse_train)
valid_seq = PremadePairSequence(valid, valid_triplet_dis, batch_size = 1024, reverse_x_in=reverse_valid)
test_seq = PremadePairSequence(test, test_triplet_dis, batch_size = 1024, reverse_x_in=reverse_test)


In [9]:
for x_batch, y_batch in train_seq:
    _, d1, d2, d3 = x_batch.shape

## Creating Model

In [10]:
model = nets.make_conv_classifier(codings_size, (d1, d2, d3), exp_filter_1d_size, filter_num=exp_filter_num)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 4, 600, 2)]       0         
                                                                 
 masking (Masking)           (None, 4, 600, 2)         0         
                                                                 
 separable_conv2d (Separable  (None, 1, 598, 4)        32        
 Conv2D)                                                         
                                                                 
 batch_normalization (BatchN  (None, 1, 598, 4)        16        
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 1, 598, 4)         0         
                                                                 
 reshape (Reshape)           (None, 598, 4)            0     

In [11]:
opt = keras.optimizers.SGD(learning_rate=learning_rate, momentum=momentum, nesterov=is_nesterov) # 0.1
model.compile(loss='mse', metrics=['accuracy', nets.crm_specificity, tf.keras.metrics.Recall(), tf.keras.metrics.Precision(), nets.crm_f1_score], optimizer=opt) #  

In [12]:
early_stopping = keras.callbacks.EarlyStopping(patience=patience, min_delta=1/100000, restore_best_weights=True, monitor=monitor, start_from_epoch = start_from_epoch, mode='max') #val_precision

In [13]:
def exponential_decay_fn(epoch, lr):
    '''
    This function decreases the learning rate according to the epoch
    '''
    return lr*0.1**(1/100)

lr_scheduler = keras.callbacks.LearningRateScheduler(exponential_decay_fn)

## Training Model

In [14]:
model.fit(train_seq, epochs=500, validation_data=valid_seq, workers=26, callbacks=[early_stopping, lr_scheduler]) #,workers=26

Epoch 1/500
36/36 [==============================] - 10s 169ms/step - loss: 0.2575 - accuracy: 0.5224 - crm_specificity: 0.5352 - recall: 0.5097 - precision: 0.5230 - crm_f1_score: 0.5159 - val_loss: 0.2503 - val_accuracy: 0.4855 - val_crm_specificity: 0.4534 - val_recall: 0.5176 - val_precision: 0.4864 - val_crm_f1_score: 0.5012 - lr: 0.0977
Epoch 2/500
36/36 [==============================] - 6s 155ms/step - loss: 0.2479 - accuracy: 0.5463 - crm_specificity: 0.5527 - recall: 0.5399 - precision: 0.5469 - crm_f1_score: 0.5427 - val_loss: 0.2485 - val_accuracy: 0.5347 - val_crm_specificity: 0.2640 - val_recall: 0.8055 - val_precision: 0.5225 - val_crm_f1_score: 0.6338 - lr: 0.0955
Epoch 3/500
36/36 [==============================] - 6s 156ms/step - loss: 0.2462 - accuracy: 0.5479 - crm_specificity: 0.5313 - recall: 0.5644 - precision: 0.5463 - crm_f1_score: 0.5548 - val_loss: 0.2462 - val_accuracy: 0.5568 - val_crm_specificity: 0.3616 - val_recall: 0.7520 - val_precision: 0.5408 - val_c

In [15]:
model.save("ExperimentAllPairModel")

INFO:tensorflow:Assets written to: ExperimentAllPairModel/assets


INFO:tensorflow:Assets written to: ExperimentAllPairModel/assets


## Evaluating Model

In [16]:
model = keras.models.load_model("ExperimentAllPairModel", custom_objects = {"crm_specificity": nets.crm_specificity, "crm_f1_score": nets.crm_f1_score})

In [17]:
nets.print_results(train_seq, model)

36/36 [==============================] - 4s 121ms/step - loss: 0.2514 - accuracy: 0.5458 - crm_specificity: 0.1806 - recall: 0.9111 - precision: 0.5265 - crm_f1_score: 0.6673
0.2516,54.45,17.75,91.15,52.57,66.68


In [18]:
nets.print_results(valid_seq, model)

7/7 [==============================] - 1s 129ms/step - loss: 0.2512 - accuracy: 0.5452 - crm_specificity: 0.1747 - recall: 0.9157 - precision: 0.5260 - crm_f1_score: 0.6682
0.252,54.36,17.63,91.08,52.51,66.62


In [19]:
nets.print_results(test_seq, model)

7/7 [==============================] - 1s 124ms/step - loss: 0.2510 - accuracy: 0.5435 - crm_specificity: 0.1733 - recall: 0.9138 - precision: 0.5250 - crm_f1_score: 0.6669
0.252,54.15,17.12,91.18,52.39,66.54
